In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
import DifferentialEquations as DE
using CairoMakie
using GLMakie
include("multiharmonic_balance.jl");

# Harmonic Balance using multiple harmonics

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

Define the variable parameters for the wave equation

In [ ]:
gamma = 0;
omega = 10;
gamma3 = 40;
g0 = 9.80665; # m / s^2
height = 5; # m
harmonics = 2; # number of harmonics

Define the constants specific to the discretizations

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nt = 5
N = 100;
order = 2;
stepx = (xright-xleft)/N;

In [ ]:
# Define symbolics
Model.@parameters x, t;

Dx = Model.Differential(x);
Dt = Model.Differential(t);

In [ ]:
u0 = ones((N + 1) * harmonics * 2);

In [ ]:
# for i in 1:1
vars, var_exprs, u = create_ansatz(x, t, omega, harmonics);
bcs = create_bcs(vars, [xleft, xright], 0);

F = 50 * exp(-40*(x^2))*sin(omega*t)

y = Dt(Dt(u)) - 9*Dx(Dx(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;

Model.@variables x, t;
expanded = expand_trig(y, x, t, omega);

eqs = make_residual(expanded, harmonics, omega, t);

solution_coeffs = solve_harmonicbalance(eqs, harmonics, bcs, [xleft, xright], stepx, x, var_exprs, u0);

u0 = vcat(solution_coeffs...);
# end

In [ ]:
solution_coeffs

In [ ]:
dt = 0.01
xgrid = collect(range(start=0.0, stop=1.0, step=stepx))

# Observable for u(x,t)
# Initialize observable with an array of zeros
u = Observable(zeros(length(xgrid)))

fig = Figure(resolution = (800, 500))
ax = Axis(fig[1, 1],
    title = "t = 0.0",
    xlabel = "x",
    ylabel = "u(x, t)",
    limits = (nothing, nothing, -0.5, 0.5)
)
lines!(ax, xgrid, u)
display(fig)

@async begin
    for k in 1:Int(Nt/dt)
        t_discrete = k * dt
        
        # Reset and compute the full sum
        u_new = zeros(length(xgrid))
        j = 1
        for i in 1:(2*harmonics)
            if isodd(i)
                u_new .+= solution_coeffs[i] .* sin(j * omega * t_discrete)
            else
                u_new .+= solution_coeffs[i] .* cos(j * omega * t_discrete)
                j += 1
            end
        end
        
        # Single assignment triggers the update
        u[] = u_new
        ax.title = "t = $(round(t_discrete, digits=2))"
        sleep(1/10)
    end
end